In [3]:
"It is the beautiful applcation  of a bayes classifier that is a much primitive algorithm in machine learning but it was first. So i took the initiative to go for this one and in future too i'll post more crazy stuff"

import pandas as pd
import numpy as np

import requests   #for url
import io


url  = "https://raw.githubusercontent.com/kartikpaliwal/Bays-classifier-Rain-prediction-multivariate/master/seattleWeather_1948-2017.csv"

s=requests.get(url).content
AllData=pd.read_csv(io.StringIO(s.decode('utf-8')))

TrainData = AllData[0:int(0.60*len(AllData))]
TestData = AllData[(int(0.60*len(AllData))):] 

TestRain = list(TestData.RAIN)

TrainData = np.array(TrainData)
TestData = np.array(TestData)

#Probability Rain
TrainData1 = []                #True
TrainData0 = []                #False
for i in range(len(TrainData)):
    if TrainData[i, 4] == 1:
        TrainData1.append(TrainData[i, 1:4])
    else:
        TrainData0.append(TrainData[i, 1:4])

TrainData1 = np.array(TrainData1)
TrainData0 = np.array(TrainData0)

PRain = len(TrainData1)/len(TrainData)
PRainNot = (1-PRain)

#MLE of mean:
def MLEmean(N, Data):
    return(sum(Data)/N)

#For True Data:
ThetaMeanTmax = MLEmean(len(TrainData1), TrainData1[:, 1])
ThetaMeanTmin = MLEmean(len(TrainData1), TrainData1[:, 2])

#For False Data:
ThetaMTmax = MLEmean(len(TrainData0), TrainData0[:, 1])
ThetaMTmin = MLEmean(len(TrainData0), TrainData0[:, 2])

#MLE of Std:
def MLEstandard(N, Data, Theta):
    BlankList=[]
    for i in Data:
        std = ((i-Theta)**2)
        BlankList.append(std)
    return(((1/(N-1))*(sum(BlankList))))

#For True Data:
ThetaStdTmax = MLEstandard(len(TrainData1), TrainData1[:, 1], ThetaMeanTmax)
ThetaStdTmin = MLEstandard(len(TrainData1), TrainData1[:, 2], ThetaMeanTmin)

#For False Data:
ThetaSTmax = MLEstandard(len(TrainData0), TrainData0[:, 1], ThetaMTmax)
ThetaSTmin = MLEstandard(len(TrainData0), TrainData0[:, 2], ThetaMTmin)



In [185]:
#Userinput PRCPi TMAXi TMINi
UserInput = (input("Enter "))
UserInput = np.array([float(i) for i in UserInput.split(' ')])
UserInput = np.transpose(UserInput)
Thetas1 = [ThetaMeanTmax, ThetaMeanTmin]
EPower = np.matrix(UserInput-Thetas1)

Enter 54 45


In [186]:
def PJoin(UInput, Mean, Variance):
    return((1/(np.sqrt(2*np.pi*Variance)))*np.exp(-(((UInput-Mean)**2)/(2*Variance))))
    
TmaxIntT = (PJoin(UserInput[0], ThetaMeanTmax, ThetaStdTmax))
TminIntT = (PJoin(UserInput[1], ThetaMeanTmin, ThetaStdTmin))

TProb = TmaxIntT* TminIntT

#PrcpIntF = .01
TmaxIntF = (PJoin(UserInput[0], ThetaMTmax, ThetaSTmax))
TminIntF = (PJoin(UserInput[1], ThetaMTmin, ThetaSTmin))

FProb = TmaxIntF* TminIntF

ConProbT = ((TProb*PRain)/((TProb*PRain)+(FProb*PRainNot)))

ConProbF = ((FProb*PRain)/((FProb*PRain)+(TProb*PRainNot)))

if ConProbT>= ConProbF:
    print("Rain True")
else:
    print("Rain False")

Rain True


In [187]:
# testing phase begins
TestT = (np.array([TestData[:, 2], TestData[:, 3]])).T

In [188]:
UserInput = []
TestP = []
for i in TestT:
    UserInput = i
    Thetas1 = [ThetaMeanTmax, ThetaMeanTmin]
    EPower = np.matrix(UserInput-Thetas1)

    TmaxIntT = (PJoin(UserInput[0], ThetaMeanTmax, ThetaStdTmax))
    TminIntT = (PJoin(UserInput[1], ThetaMeanTmin, ThetaStdTmin))

    TProb = TmaxIntT* TminIntT

    #PrcpIntF = .01
    TmaxIntF = (PJoin(UserInput[0], ThetaMTmax, ThetaSTmax))
    TminIntF = (PJoin(UserInput[1], ThetaMTmin, ThetaSTmin))

    FProb = TmaxIntF* TminIntF

    ConProbT = ((TProb*PRain)/((TProb*PRain)+(FProb*PRainNot)))

    ConProbF = ((FProb*PRain)/((FProb*PRain)+(TProb*PRainNot)))

    if ConProbT>= ConProbF:
        TestP.append(1)
    else:
        TestP.append(0)
 
AccuracyTrue = 0
for i in range(len(TestP)):
    if TestP[i] == TestRain[i]:
        AccuracyTrue+=1

Accuracy = (AccuracyTrue/len(TestT))*100    

In [189]:
Accuracy

66.9406124645338